1. dataset lavarage , divisione training set/ test set
2. trovo tag e generi preferiti (loro hanno fatto 3 per visualizzazione)
3. trovo media voti per generi e tag per ogni utente
4. clustering con 7 metodi diversi(tengo conto solo di utenti che apprezzano almeno uno dei generi/tag)
5. prediction based on similar users ratings (top N-movies)
6. evaluation of the model created with different criteria
7. relazione

In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

movies = pd.read_csv('/content/movies.csv')

ratings = pd.read_csv('/content/ratings.csv')

tags = pd.read_csv('/content/tags.csv')



#Data inspection

In [53]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),NaN
1,2,Jumanji (1995),NaN
2,3,Grumpier Old Men (1995),NaN
3,4,Waiting to Exhale (1995),NaN
4,5,Father of the Bride Part II (1995),NaN


In [42]:
ratings = ratings.drop('timestamp' , axis=1)
ratings.describe()

,userId,movieId,rating
count,100836.000000,100836.000000,100836.000000
mean,326.127564,19435.295718,3.501557
std,182.618491,35530.987199,1.042529
min,1.000000,1.000000,0.500000
25%,177.000000,1199.000000,3.000000
50%,325.000000,2991.000000,3.500000
75%,477.000000,8122.000000,4.000000
max,610.000000,193609.000000,5.000000


In [43]:
tags = tags.drop('timestamp', axis=1)
tags.describe()

,userId,movieId
count,3683.000000,3683.000000
mean,431.149335,27252.013576
std,158.472553,43490.558803
min,2.000000,1.000000
25%,424.000000,1262.500000
50%,474.000000,4454.000000
75%,477.000000,39263.000000
max,610.000000,193565.000000


#Favourite genres and Tags

In [60]:
movies['genres'] = movies['genres'].str.split('|')
exploded_movies = movies.explode('genres')

diff_genres = (set(np.array(exploded_movies['genres'])))
print(diff_genres)
print('Number of different genres: ', len(diff_genres))

{'Crime', 'Film-Noir', 'Romance', '(no genres listed)', 'Thriller', 'Western', 'Children', 'Adventure', 'Mystery', 'Fantasy', 'War', 'Horror', 'Documentary', 'Sci-Fi', 'Drama', 'Comedy', 'Musical', 'IMAX', 'Animation', 'Action'}
Number of different genres:  20


In [95]:
diff_genres = list(diff_genres)
print(diff_genres)
pos_ratings = {}

for genre in diff_genres:
  set_of_movies = exploded_movies[exploded_movies['genres'] == genre]
  Ids = set(set_of_movies['movieId'])
  ratings_by_genre = ratings[ratings['movieId'].isin(Ids)]
  positive_ratings = len(ratings_by_genre[ratings_by_genre['rating']>3.5])
  avg_rating = ratings_by_genre['rating'].mean()
  pos_ratings[genre] = positive_ratings

  print('\ngenre: ' + genre)
  print('Number of Movies: ',len(Ids))
  print('avg rating: ', avg_rating)
  print('number of ratings: ',len(ratings_by_genre))
  print('number of positive ratings: ',positive_ratings)
  #ordering of pos_ratings
print(pos_ratings)

['Crime', 'Film-Noir', 'Romance', '(no genres listed)', 'Thriller', 'Western', 'Children', 'Adventure', 'Mystery', 'Fantasy', 'War', 'Horror', 'Documentary', 'Sci-Fi', 'Drama', 'Comedy', 'Musical', 'IMAX', 'Animation', 'Action']

genre: Crime
Number of Movies:  1199
avg rating:  3.658293867274144
number of ratings:  16681
number of positive ratings:  9077

genre: Film-Noir
Number of Movies:  87
avg rating:  3.920114942528736
number of ratings:  870
number of positive ratings:  588

genre: Romance
Number of Movies:  1596
avg rating:  3.5065107040388437
number of ratings:  18124
number of positive ratings:  8668

genre: (no genres listed)
Number of Movies:  34
avg rating:  3.4893617021276597
number of ratings:  47
number of positive ratings:  23

genre: Thriller
Number of Movies:  1894
avg rating:  3.4937055799183425
number of ratings:  26452
number of positive ratings:  12537

genre: Western
Number of Movies:  167
avg rating:  3.583937823834197
number of ratings:  1930
number of positiv